Deving modification locator script.

evidence.txt is huge.

In [1]:
wc -l evidence.txt

4700965 evidence.txt


Only using 2 columns. The modified sequence and the leading razor protein.

In [2]:
awk -F"\t" '{print $4,$15}' evidence.txt | head

Modified sequence Leading razor protein
_AAAAAAAAAAAAAAAGAGAGAK_ tr|Q53ZR1|Q53ZR1_HUMAN
_AAAAAAAAAAAAAAAGAGAGAK_ tr|Q53ZR1|Q53ZR1_HUMAN
_AAAAAAAAAAAAAAAGAGAGAK_ tr|Q53ZR1|Q53ZR1_HUMAN
_AAAAAAAAAAAAAAAGAGAGAK_ tr|Q53ZR1|Q53ZR1_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN


Assuming the queried amino acid is Q

In [3]:
awk -F"\t" '($4 ~ /Q/){print $4,$15}' evidence_head.txt 

_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN


Let's find a bunch with some modifications and a Q and R in it. (uniq doesn't like being cut off so 2>/dev/null it)

In [16]:
awk -F"\t" '($4 ~ /\).*Q.*R.*\)/){print $4,$15}' evidence.txt | uniq 2>/dev/null | head 

_(ac)AAAAVQGGRS(ph)GGSGGCSGAGGASNCGTGSGR_ tr|E9PL01|E9PL01_HUMAN
_(ac)AAAETQSLREQPEM(ox)EDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAETQSLREQPEMEDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAETQS(ph)LREQPEMEDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAET(ph)QSLREQPEMEDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAETQS(ph)LREQPEMEDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAPQAPGRGS(ph)LR_ sp|P78345|RPP38_HUMAN
_(ac)AAAPQAPGRGS(ph)LRK_ sp|P78345|RPP38_HUMAN
_(ac)AAARPEAQSRS(ph)SPTPESR_ sp|Q8NCA9|ZN784_HUMAN
_(ac)AAARPEAQSRSS(ph)PTPESR_ sp|Q8NCA9|ZN784_HUMAN


In this list, T and M are modified once so let's use them as our amino acids.